# Road Following - Build TensorRT model for live demo

In this notebook, we will optimize the model we trained using TensorRT.

## Load the trained model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebook's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [1]:
import torchvision
import torch
from jetbot.utils import model_selection
import os 
import ipywidgets.widgets as widgets
from ipywidgets.widgets import Box, HBox, VBox, Layout, Label
import traitlets

model_repo_dir = os.environ['MODEL_REPO_DIR_DOCKER']
print(model_repo_dir)

# pth_ms = model_selection(core_library = "Pytorch", model_repo_dir=model_repo_dir)
pth_ms = model_selection(core_library = "Pytorch")

/workspace/model_repo


In [2]:
pth_ms.model_function = "classifier"

model_type_widget = widgets.Select(options=pth_ms.model_type_list, value=pth_ms.model_type_list[0],
                                      description='Model Type:')
traitlets.dlink((pth_ms, 'model_type_list'), (model_type_widget, 'options'))
traitlets.dlink((model_type_widget, 'value'), (pth_ms, 'model_type'))
# traitlets.dlink((pth_ms, 'model_type'), (RC, 'type_cruiser_model'))

model_path_widget = widgets.Select(options=pth_ms.model_path_list, description='Model Path:',
                                      layout=Layout(width='60%'))
traitlets.dlink((pth_ms, 'model_path_list'), (model_path_widget, 'options'))
traitlets.dlink((model_path_widget, 'value'), (pth_ms, 'model_path'))
# traitlets.dlink((pth_ms, 'model_path'), (RC, 'cruiser_model'))

display(HBox([model_type_widget, model_path_widget]))

In [3]:
model_path = model_path_widget.value
pth_model_name = model_path_widget.value.split('/')[-1].split('.')[0].split('_', 4)[-1]
print(pth_model_name, '\n', model_path)

efficientnet_b4 
 /workspace/model_repo/road_following/best_steering_model_xy_efficientnet_b4.pth


In [4]:
import torchvision.models as pth_models
model = getattr(pth_models, pth_model_name)()

# ----- modify last layer for classification, and the model used in notebook should be modified as well.

if pth_model_name == 'mobilenet_v3_large':  # MobileNet
    model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, 2)  # for mobilenet_v3 model. must add the block expansion factor 4

elif pth_model_name == 'mobilenet_v2':
    model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 2)  # for mobilenet_v2 model. must add the block expansion factor 4

elif pth_model_name == 'vgg11': # VGGNet
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)  # for VGG model. must add the block expansion factor 4

elif 'resnet' in pth_model_name: # ResNet
    model.fc = torch.nn.Linear(model.fc.in_features, 2)  # for resnet model must add the block expansion factor 4
    # model.fc = torch.nn.Linear(512, 2)

elif pth_model_name == 'inception_v3':   # Inception_v3
    model.fc = torch.nn.Linear(model.fc.in_features, 2)
    if model.aux_logits:
        model.AuxLogits.fc = torch.nn.Linear(model.AuxLogits.fc.in_features, 2)

elif 'efficientnet' in pth_model_name:   # efficientnet
    model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 2)  # for efficientnet_b1 

model = model.cuda().eval().half()

Next, load the trained weights from the ``best_steering_model_xy.pth`` file that you uploaded.

In [5]:
model.load_state_dict(torch.load(model_path))
# model.load_state_dict(torch.load('best_steering_model_xy_resnet18.pth'))
# model.load_state_dict(torch.load('best_steering_model_xy_resnet34.pth'))
# model.load_state_dict(torch.load('best_steering_model_xy_mobilenet_v3_large.pth'))
# model.load_state_dict(torch.load('best_steering_model_xy_inception_v3.pth'))

<All keys matched successfully>

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [6]:
device = torch.device('cuda')

## TensorRT

> If your setup does not have `torch2trt` installed, you need to first install `torch2trt` by executing the following in the console.
```bash
cd $HOME
git clone https://github.com/NVIDIA-AI-IOT/torch2trt
cd torch2trt
sudo python3 setup.py install
```

Convert and optimize the model using torch2trt for faster inference with TensorRT. Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete.

In [7]:
from torch2trt import torch2trt

if pth_model_name == 'inception_v3':
    data = torch.zeros((1, 3, 299, 299)).cuda().half()   # inception_v3
else:
    data = torch.zeros((1, 3, 224, 224)).cuda().half()  # resnet

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [8]:
path_trt_model = os.path.join(model_repo_dir, 'road_following', "best_steering_model_xy_trt_"+pth_model_name+'.pth')
torch.save(model_trt.state_dict(), path_trt_model)
# torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_resnet18.pth')
# torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_resnet34.pth')
# torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_mobilenet_v3_large.pth')
# torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_inception_v3.pth')

## Next
Open live_demo_trt.ipynb to move JetBot with the TensorRT optimized model.